# Reinforcement Learning Based Cognitive Radio Project

*by Leif Skunberg, Michael Lyons, and Charles Culpepper, December, 2017*

## Introduction

#### Artificial Intelligence

Artificial intelligence, and the concept of a cognitive radio, have both been around for quite a while. Machines have been learning since at least 1950, with Alan Turing’s aptly named Learning Machine. But the mathematical foundations come from Bayes Theroem and the concept of least squares, which originates in the early 1800s. 
Artificial intelligence, as the name implies, is the creation of a synthetic form of intelligence. The goal is to create a machine that can learn. While we started with Turing’s simple machine, we would have a form of neural networks a couple of years later. The concept has been studied and abstracted almost constantly since then. Today, machine learning and artificial intelligence are one of the hottest trends in technology. A lot of the math that wasn’t worked out in the fifties, is readily available in 2017. The computers today are also infinitely more powerful, and can perform calculations in real time that early computers couldn’t process at all. These developments bring the possibility of a true cognitive radio closer to a realistic possibility

#### Cognitive Radio

Radio is ubiquitous in our society. Radio waves cover our globe, and provide everything from standard communication, to music in your car, to the frequencies used to provide the world with wireless internet. Radio waves are a type of electromagnetic radiation at a given range of frequencies. There are other waves in this spectrum, depending on the frequency. These include everything from x-rays and infrared, to UV and standard light. Currently, you tune a radio to a frequency, and it will pick up any communications on that channel. The stronger your radio, the more of the range it can access and transmit through. Some frequencies are off limits for civilians, and some others are used for specific services, like wi-fi. But within the standard range, most people know what it means to tune a radio to a frequency – the number on your favorite radio station, is a frequency.

In addition to simply tuning into the right frequency, there are several components to radio communications, the most obvious of which is interference. As you drive out of your local area, you may notice your radio station start to pick up another, or maybe die off into static. Either way, you’re seeing a form of interference. Ideally, if you can stay on the channels with the least interference, you’ll always have a clear line of communication.

A cognitive radio is one that can see, and over time, learn, which frequencies have the least interference at a given time, and can hop to the channel with least interference so that the communication stays clear. Basically, a cognitive radio is one that switches frequencies for you, depending on what it thinks is the clearest line of communication. At it’s most basic, it can simply decide between lines that are open, and lines that are already in use, keeping the user on the open lines as much as possible. This frequency hopping is called Frequency Hopping Spread Spectrum (FHSS).

The idea for a cognitive radio started back in 1998, where Joseph Mitola III made a proposal for it at a seminar for KTH (translates to Royal Institute of Technology in Stockholm). There is no fully working example of a live cognitive radio to date. Rather, it is an established concept that is accepted as possible, but is still a goal that the industry is actively working toward.
The latest trends in AI are what makes advances in cognitive radio possible. For a radio receiver to know which lines are open (or which have a satisfactory level of interference), it needs to know, compute, remember, and learn about the elements included therein. This necessitates some form of machine learning. As our technology develops, we are getting closer to processing the needed calculations in real time, meaning, as the communication happens.
Currently, most experiments regarding cognitive radio take place either with sample data, or with captured data from a live radio. While many have explored both in depth, no one has been able to perform the functionality as the communication happens.

#### Cognitive Radio Experiment in Reinforcement Learning

These same limitations impact our own experiment. Due to time constraints, we chose to run our experiment with sample data. The idea is that you should be able to apply reinforcement learning to FHSS using GNURadio, creating a software based cognitive radio. The intent is also to encrypt the message, and so a final version would be able to send end-to-end secure communications, over radio, on the clearest lines available, with no message deterioration. While we were unable to perform the functions on live data, we created sample data and a reinforcement learning algorithm in order to make a proof of concept. We are including RSA encryption code so as to have all the parts of the model we are exploring.

#### RSA Encryption

There are many ways to encrypt and decrypt data for preparation to transmit it in a radio signal. We chose an asymmetric form named RSA. The basic functionality of an asymmetric cryptographic system is the existence of two keys, a public key and a private key. It makes more intuitive sense if you consider the public key a lock. Anyone can lock something with it, but only the owner of the private key can unlock those locks. 
A user would typically use the public key of their planned recipient to encrypt some data.That data can then be encapsulated in another encryption, created by the user’s own public key. Anyone can unlock the outer layer of encryption because it uses a public key, but the inner one still needs the corresponding private key. The point of this is to prove that the sender is authentic. That user’s public key, while public, is unique, and proves that it was encrypted by the original sender. The recipient then receives the communication, uses the public key to decrypt the outer layer, and his own private key to decrypt the inner layer, and get the data. That ensures that the sender is authentic, the message is secure, and the receiver is the intended person. This system provides end to end security for any data that can be stored and transmitted digitally.


## Methods

Steps I took.  Resources I used, such as our textbook [Russell and Norvig, 2014]. 

Say in detail what each team member did.

## Results

Show all results.  Intermediate resultw might be shown in above Methods section.  Plots, tables, whatever.

## Conclusions

What I learned.

### References

* [Russell and Norvig, 2014] Stuart Russell and Peter Norvig, [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/), Publisher. 2014.
* [chuck] me, fldfkafjlak

Your report should contain approximately 1,500 to 5,000 words, in markdown cells.  You can count words by running the following python code in your report directory.

In [2]:
import io
from nbformat import current
import glob
nbfile = glob.glob('*.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Example of Project Report.ipynb is 123
